# Linear programming

Linear programming tends to be the first place that teaching for OR begins.

In [4]:
import scipy.optimize as opt

## Basic elements

We compose the mathematical program out of:

1. The objective function – e.g. $$\min f(x_1, x_2, \ldots, x_n)$$
2. The constraints – e.g. $$s.t.\ \ g_i(x_1, x_2, \ldots,x_n) \le b_i\ \ \forall i=1, \ldots, m$$
3. The decision variable – e.g. $$x_j \isin \mathbb{R}\ \ \forall j = 1, \ldots , n$$

(The decision variable is the actual decision we make – numbers of items, yes or no, etc.)

$x$ is going to take the format of a vector of decision variables, or our *decision vector*, e.g. $$x = \begin{bmatrix}x_{1} \\
\vdots \\
x_{n}\end{bmatrix}$$

A mathematical program is a LP if $f$ and $g$ are all **linear** functions.

### Linear functions

Each linear function may be expressed as $$a_1x_1 + a_2x_2 + \cdots + a_nx_n = \sum_{j=1}^n a_jx_j$$

## Three types

Any LP must be:
* Infeasible (i.e. its feasible region is empty)
* Unbounded (i.e. for any feasible solution, there is another feasible solution that is better)
* Finitely optimal (i.e. it has an optimal solution)

A finitely optimal LP may have:
* A unique optimal solution
* Multiple optimal solutions

## Transforming minimisation to maximisation

If ever we need to reframe a minimisation problem as a maximisation problem, it is relatively straightforward:

$$ max\ f(x) \Leftrightarrow min -f(x)$$

And to adapt our constraints:

$$ max\ \ x_1 - x_2 \\
s.t.\ \ -2x_1 + x_2 \ge -3 \\
x_1 + 4x_2 = 5 \\
\ \\
\Leftrightarrow \\
\ \\
min\ \ -x_1 + x_2 \\
s.t.\ \ 2x_1 - x_2 \le 3 \\
x_1 + 4x_2 \le 5 \\
-x_1 - 4x_2 \le -5 $$

Focusing on that equality equivalent:

$$g_i(x) = b_i \Leftrightarrow g_i(x) \le b_i, g_i(x) \le -b_i$$

## Constraints

We can break down two types of constraints:

* Sign constraints, e.g. $x_i \ge 0$, or $x_i \le 0$
* Or a functional constraint (all others)

At a solution, a constraint is binding if:

Let $g(\cdot) \le b$ be an inequality constraint, and $\bar{x}$ be a solution. $g(\cdot) \le b$ is binding at $\bar{x}$ if $g(\bar{x}) = b$.

(An equality constraint is always binding at any feasible solution.)

## Problem formulation

* We produce desks and tables.
    * Producing a desk requires three units of wood, one hour of labor, and 50 minutes of machine time.
    * Producing a table requires five units of wood, two hours of labor, and 20 minutes of machine time.
* We can sell everything we produce.
* Each day, we have:
    * Two hundred workers, each working for eight hours
    * 50 machines that run for sixteen hours
    * 3600 units of wood
* Desks and tables are sold for $700 and $900 per unit, respectively.

Let $x_1$ be how many desks we sell, and $x_2$ be how many tables we sell.

Therefore our problem becomes:

$$
max (700x_1 + 900x_2) \\
s.t. \\
3x_1 + 5x_2 \le 3600 \\
1x_1 + 2x_2 \le 200\times8 \\
50x_1 + 20x_2 \le 50\times16\times60

$$

Or, framed in terms of minimisation:

$$
min (-700x_1 + -900x_2) \\
s.t. \\
3x_1 + 5x_2 \le 3600 \\
1x_1 + 2x_2 \le 200\times8 \\
50x_1 + 20x_2 \le 50\times16\times60

$$

And of course:

$$
x_1, x_2 \ge 0
$$

We solve this in SciPy, noting that all inequalities must be framed in terms of "less than equals":

In [6]:
opt.linprog(
    c=[-700, -900],
    A_ub=[
        [3, 5],
        [1, 2],
        [50, 20]
    ],
    b_ub=[
        3600,
        200*8,
        50*16*60
    ],
    bounds=[
        [0, None],
        [0, None]
    ]
)

     con: array([], dtype=float64)
     fun: -789473.6299514732
 message: 'Optimization terminated successfully.'
     nit: 5
   slack: array([2.47383124e-04, 3.36842192e+02, 3.30042907e-03])
  status: 0
 success: True
       x: array([884.2104655 , 189.47367122])

As a follow-up, consider the following:

* We produce and sell a product.
* For the coming four days, we must fulfill the following demands:
    * 100, 150, 250, and 170 units for days 1, 2, 3, and 4, respectively.
* Unit production costs are different for different days.
    * $9, $12, $10, and $12 for each day, respectively.
* The prices are fixed, so maximising profits is the same as minimising costs.
* We can store a product and sell it later, but the inventory cost is $1 a day.

If we assume that we won't have any inventory costs on day 4:

In [ ]:
opt.linprog(
    c=[(9+3), (12+2), (10+1), (12+0)],
    A_ub=[
        [-1, 0, 0, 0],
        [-1, -1, 0, 0],
        [-1, -1, -1, 0],
        [-1, -1, -1, -1],
    ],
    b_ub=[
        -(100),
        -(100+150),
        -(100+150+200),
        -(100+150+200+170)
    ],
    bounds=[
        [0, None],
        [0, None],
        [0, None],
        [0, None],
    ]
)

     con: array([], dtype=float64)
     fun: 7069.999999885983
 message: 'Optimization terminated successfully.'
     nit: 6
   slack: array([ 1.50000000e+02, -3.02472358e-09,  1.70000000e+02, -1.05836762e-08])
  status: 0
 success: True
       x: array([2.50000000e+02, 1.94070197e-10, 3.70000000e+02, 5.03929233e-09])

Note the slack that we have – the costs of production + storage are effectively the same at this point.

If we don't assume that our inventory costs stop at day 3...

In [30]:
opt.linprog(
    c=[(9+4), (12+3), (10+2), (12+1)],
    A_ub=[
        [-1, 0, 0, 0],
        [-1, -1, 0, 0],
        [-1, -1, -1, 0],
        [-1, -1, -1, -1],
    ],
    b_ub=[
        -(100),
        -(100+150),
        -(100+150+200),
        -(100+150+200+170)
    ],
    bounds=[
        [0, None],
        [0, None],
        [0, None],
        [0, None],
    ]
)

     con: array([], dtype=float64)
     fun: 7689.999999836162
 message: 'Optimization terminated successfully.'
     nit: 6
   slack: array([ 1.50000000e+02, -3.76186904e-09,  1.70000000e+02, -1.38753649e-08])
  status: 0
 success: True
       x: array([2.50000000e+02, 2.49332231e-10, 3.70000000e+02, 5.92945679e-09])

It's the same, goodness!